In [2]:
import os
import json
import numpy as np
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix

# Paths to your data directories
train_path = r"H:\2.vgg19 image procced\preprocced_data_with_split\train"
test_path = r"H:\2.vgg19 image procced\preprocced_data_with_split\test"
val_path = r"H:\2.vgg19 image procced\preprocced_data_with_split\val"
IMG_SIZE = (224, 224)  # Adjusted for VGG19 input size

# Directory to save models and history
save_dir = r"H:\1.organized\allModel"

# Set hyperparameters
batch_size = 32
num_classes = 105
epochs = 5

# Create data generators for image preprocessing and augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

# Data generators for training, testing, and validation data
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=IMG_SIZE,
    batch_size=batch_size,
    class_mode='categorical'
)
test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=IMG_SIZE,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False  # Keep the order for evaluation
)
validation_generator = test_datagen.flow_from_directory(
    val_path,
    target_size=IMG_SIZE,
    batch_size=batch_size,
    class_mode='categorical'
)

# Load the pre-trained VGG19 model
base_model_vgg19 = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers except the last few layers
for layer in base_model_vgg19.layers[:-3]:
    layer.trainable = False

# Create a custom top classifier for the number of classes using VGG19
model_vgg19 = Sequential()
model_vgg19.add(base_model_vgg19)
model_vgg19.add(Flatten())
model_vgg19.add(Dense(2048, activation='relu'))
model_vgg19.add(Dropout(0.5))  # Adjust dropout as needed
model_vgg19.add(Dense(num_classes, activation='softmax'))

model_vgg19.summary()

# Compile VGG19 model
model_vgg19.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Define the checkpoint callback to save the model
checkpoint_path = os.path.join(save_dir, '2.vgg19_modeltry.h5')
checkpoint = ModelCheckpoint(
    filepath=checkpoint_path,
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)

# Train the model and save the history
history_vgg19 = model_vgg19.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    callbacks=[checkpoint]
)

# Save model architecture to JSON (for future use in prediction)
model_json = model_vgg19.to_json()
with open(os.path.join(save_dir, '2.model_vgg19try.json'), 'w') as json_file:
    json_file.write(model_json)

# Save label mapping as JSON (assuming you have train_generator.class_indices)
label_map = train_generator.class_indices
with open(os.path.join(save_dir, '2.vgg19label_maptry.json'), 'w') as json_file:
    json.dump(label_map, json_file)

# Evaluate the model on the test set
test_loss, test_accuracy = model_vgg19.evaluate(test_generator, verbose=1)
print(f"Test Accuracy: {test_accuracy}")

# Make predictions on the test set
predictions = model_vgg19.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)

# Get true labels from the generator
true_classes = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

# Generate a classification report and confusion matrix
print("Classification Report:")
print(classification_report(true_classes, predicted_classes, target_names=class_labels))

print("Confusion Matrix:")
cm = confusion_matrix(true_classes, predicted_classes)
print(cm)


Found 8725 images belonging to 105 classes.
Found 1837 images belonging to 105 classes.
Found 1972 images belonging to 105 classes.
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 7, 7, 512)         20024384  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 2048)              51382272  
                                                                 
 dropout (Dropout)           (None, 2048)              0         
                                                                 
 dense_1 (Dense)             (None, 105)               215145    
                                                                 
Total params: 71,621,801
Trainable params: 56,317,033
No

In [5]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg19 import preprocess_input
import os
from PIL import Image

# Load your VGG19 model and label mapping
save_dir = r"H:\1.organized\allModel"
model_path = os.path.join(save_dir, '2.vgg19_modeltry.h5')
label_map_path = os.path.join(save_dir, '2.vgg19label_maptry.json')

# Load model and class labels
model = load_model(model_path)
with open(label_map_path, 'r') as f:
    label_map = json.load(f)
    class_labels = {v: k for k, v in label_map.items()}

# Function to preprocess image for prediction
def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    return img_array

# Function to handle image prediction
def predict_image():
    file_path = filedialog.askopenfilename()
    if file_path:
        try:
            # Preprocess the image
            img_array = preprocess_image(file_path)

            # Make prediction
            prediction = model.predict(img_array)
            predicted_class = np.argmax(prediction)
            class_name = class_labels[predicted_class]

            # Display prediction
            result_label.config(text=f"Prediction: {class_name}")
            
            # Display the image
            img = Image.open(file_path)
            img = img.resize((300, 300), Image.ANTIALIAS)
            img = ImageTk.PhotoImage(img)
            panel.configure(image=img)
            panel.image = img  # keep a reference
            
        except Exception as e:
            messagebox.showerror("Error", f"Error predicting image: {e}")

# Create main application window
root = tk.Tk()
root.title("Image Classifier")

# Create GUI components
btn_load = tk.Button(root, text="Load Image", command=predict_image)
btn_load.pack(pady=20)

result_label = tk.Label(root, text="Prediction: ")
result_label.pack()

panel = tk.Label(root)  # for displaying the loaded image
panel.pack(padx=10, pady=10)

# Run the main event loop
root.mainloop()


1/1 [==============================] - 0s 17ms/step
